In [1]:
import sys
from PyQt5 import QtWidgets ,QAxContainer

In [2]:
class MyWindow(QtWidgets.QMainWindow):
    def __init__(self):
        super().__init__()

        self.kiwoom = QAxContainer.QAxWidget("KHOPENAPI.KHOpenAPICtrl.1")
        self.kiwoom.dynamicCall("CommConnect()")
        self.kiwoom.OnEventConnect.connect(self.event_connect)        
        self.kiwoom.OnReceiveTrData.connect(self.receive_trdata)
        
        self.setWindowTitle("PyStock")
        self.setGeometry(50,350,400,400)

        label = QtWidgets.QLabel('종목코드: ',self)
        label.move(20,20)
        
        self.code_edit = QtWidgets.QLineEdit(self)
        self.code_edit.move(80,20)
        self.code_edit.setText("039490")
        
        btn1 = QtWidgets.QPushButton("조회", self)
        btn1.move(190,20)
        btn1.clicked.connect(self.btn1_clicked)
        
        self.text_edit = QtWidgets.QTextEdit(self)
        self.text_edit.setGeometry(10,60,280,80)
        self.text_edit.setEnabled(False)
        
        btn2 = QtWidgets.QPushButton("계좌번호",self)
        btn2.move(190,140)
        btn2.clicked.connect(self.btn2_clicked)

        #Open API+는 통신연결 상태가 바뀔 때 OnEventConnect 이벤트가 발생하는 것을 이용
        
    def event_connect(self, err_code):
        if err_code ==0:
            self.text_edit.append("Login Success!")
    def btn1_clicked(self):
        code = self.code_edit.text()
        self.text_edit.append("Share Code: "+code)
        #input value
        self.kiwoom.dynamicCall("SetInputValue(QString,QString)","종목코드",code)
        self.kiwoom.dynamicCall("CommRqData(QString,QString,int,QString)","opt10001_req","opt10001",0,"0101")
    def receive_trdata(self, screen_no, rqname, trcode, recordename, prev_next, data_len,err_code, msg1,msg2):
        if rqname == "opt10001_req":
            name = self.kiwoom.dynamicCall("CommGetData(Qstring,Qstring,QString,int,QString)",trcode,"",rqname,0,"종목명")
            volume = self.kiwoom.dynamicCall("CommGetData(Qstring,Qstring,QString,int,QString)",trcode,"",rqname,0,"거래량")
            self.text_edit.append("Share Name: "+name.strip())
            self.text_edit.append("Trade Volume: "+volume.strip())
    def btn2_clicked(self):
        accountNum = self.kiwoom.dynamicCall("GetLoginInfo(QString)",["ACCNO"])
        self.text_edit.append("Account Number: "+accountNum)#.rstrip(';'))

U can make 32-bit environment in 64-bit Windows!!! Thx god.. follow next 2 urls.

http://sharebook.kr/pages/viewpage.action?pageId=9437187 &
https://stackoverflow.com/questions/33709391/using-multiple-python-engines-32bit-64bit-and-2-7-3-5

In [3]:
if __name__ =="__main__":
    app = QtWidgets.QApplication(sys.argv)
    myWindow = MyWindow()
    myWindow.show()
    app.exec_()